# The Impact of Lower Interest Rates in Brazilian Stock Market

*By Daniel Deutsch, José Lucas Barretto, Kevin kuhl and Lucas Miguel Agrizzi*

In [29]:
import os
import time
from datetime import date

import matplotlib.pyplot as plt
import pandas as pd
import requests
import yfinance as yf
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [3]:
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Data Collection

In this stage, we want to collect the data necessary for our analysis. In our case, we must collect two kinds of data: Indexes of (which kind? why?) and stock information about the indexes.

## Indexes Collection

Indexes are basically .... (explicar o que são indexes e porque eles serão utilizados na nossa análise).

Given that, the group decided to understand the effects of the index rate on the following indexes (because we considered them the most influential in the brazillian market):

| Index Acronym | Index Name                  | URL           | Sector  |
|:------------- |:----------------------------|:--------------|:-------|
| ICON          | Índice de Consumo           | right-aligned | $1600 |
| IEEX          | Índice de Energia Elétrica  | centered      |   $12 |
| IFNC          | Índice Financeiro           | are neat      |    $1 |
| IMAT          | Índice de Materiais Básicos | are neat      |    $1 |
| IMOB          | Índice Imobiliário          | are neat      |    $1 |
| INDX          | Índice Industrial           | are neat      |    $1 |
| UTIL          | Índice Utilidade Pública    | are neat      |    $1 |

In [ ]:
indexes = [
    { 'index': "ICON", 'url_code': "klDT04" },
    { 'index': "IEEX", 'url_code': "klFRVg" },
    { 'index': "IFNC", 'url_code': "klGTkM" },
    { 'index': "IMAT", 'url_code': "klNQVQ" },
    { 'index': "IMOB", 'url_code': "klNT0I" },
    { 'index': "INDX", 'url_code': "klORFg" },
    { 'index': "UTIL", 'url_code': "lVUSUw" }
]

years = [
    { 'year': "2021", 'url_code': "IwMjE" },
    { 'year': "2020", 'url_code': "IwMjA" },
    { 'year': "2019", 'url_code': "IwMTk" },
    { 'year': "2018", 'url_code': "IwMTg" },
    { 'year': "2017", 'url_code': "IwMTc" },
    { 'year': "2016", 'url_code': "IwMTY" },
    { 'year': "2015", 'url_code': "IwMTU" },
    { 'year': "2014", 'url_code': "IwMTQ" },
    { 'year': "2013", 'url_code': "IwMTM" },
    { 'year': "2012", 'url_code': "IwMTI" },
    { 'year': "2011", 'url_code': "IwMTE" },
    { 'year': "2010", 'url_code': "IwMTA" },
    { 'year': "2009", 'url_code': "IwMDk" },
    { 'year': "2008", 'url_code': "IwMDg" },
    { 'year': "2007", 'url_code': "IwMDc" },
    { 'year': "2006", 'url_code': "IwMDY" },
    { 'year': "2005", 'url_code': "IwMDU" },
    { 'year': "2004", 'url_code': "IwMDQ" },
    { 'year': "2003", 'url_code': "IwMDM" },
    { 'year': "2002", 'url_code': "IwMDI" },
    { 'year': "2001", 'url_code': "IwMDE" },
    { 'year': "2000", 'url_code': "IwMDA" },
    { 'year': "1999", 'url_code': "E5OTk" },
    { 'year': "1998", 'url_code': "E5OTg" }
]

url_base = "https://sistemaswebb3-listados.b3.com.br/indexStatisticsProxy/IndexCall/GetPortfolioDay/eyJpbmRleCI6I%siLCJsYW5ndWFnZSI6InB0LWJyIiwieWVhciI6Ij%sifQ=="

df = pd.DataFrame(columns=['date', 'index', 'rate_value'])
for i_id, index in enumerate(indexes):
    for y_id, year in enumerate(years):
        #time.sleep(2)
        r = requests.get(url_base % (index['url_code'], year['url_code']), verify=False)
        try:
            print(f"\rIndexes: {i_id+1}/{len(indexes)} Years: {y_id+1}/{len(years)}", end="")
            r = r.json()
        except Exception:
            print(f"Error with index {index['index']} year {year['year']}")
        else:
            if r['results']:
                for result in r['results']:
                    day = result['day']
                    for key, val in result.items():
                        if key != "day":
                            df = pd.concat(
                                [
                                    df, 
                                    pd.DataFrame({
                                        'date': f"{year['year']}-{int(key[9:]):02d}-{day:02d}", 
                                        'index': [index['index']], 
                                        'rate_value': [float(val.replace('.', '').replace(',', '.'))] if val else [val]
                                    })
                                ], 
                                ignore_index=True
                            )

df.dropna(inplace=True)
df.sort_values(by=['date', 'index', 'rate_value'], inplace=True, ignore_index=True)
df.to_csv("./datasets/indexes.csv.zip")

### Macroeconomic Indicators

In [82]:
url_selic = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=csv'

selic_df = pd.read_csv(url_selic,
                       header=0,
                       names = ['date', 'pct_value'],
                       delimiter=';',
                       index_col='date',
                       decimal=',',
                       parse_dates=[0])

In [139]:
url_ipca = 'https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/all/p/all/d/v63%202,v69%202,v2266%2013,v2263%202,v2264%202,v2265%202?formato=json'

ipca_df = pd.read_json(url_ipca)
ipca_df.drop([0], inplace = True)

mask = (ipca_df['MN'] == '%')
ipca_df = ipca_df[mask]

ipca_df.rename(columns={'D3C': 'date'}, inplace=True)
ipca_df['date'] = pd.to_datetime(ipca_df['date'], format='%Y%m')
ipca_df['V'] = pd.to_numeric(ipca_df['V'])
ipca_df.set_index('date', inplace=True)

mask = (ipca_df['D2N'] == 'IPCA - Variação mensal')
ipca_pct_change = ipca_df[mask][['V']].rename(columns={'V': 'pct_change'})

mask = (ipca_df['D2N'] == 'IPCA - Variação acumulada no ano')
ipca_annual_cumulative = ipca_df[mask][['V']].rename(columns={'V': 'cumulative_change'})

ipca_df = pd.concat([ipca_pct_change, ipca_annual_cumulative], axis=1)

# Data Analysis